In [1]:
# This notebook is to create different customer profiling variables 

In [2]:
# This includes re-running the queries created in the previous Exercises,
# using the data refined as requested by CFO (excluding transactions from 
# customers who had less than 5 transactions

In [3]:
# Content List
#
# 01. Importing libraries and data
# 02. Analyzing the spending pattern by region
# 02.01. Creating Region column by merging
# 02.02. Grouping values (crosstab)
# 03. Excluding low-activity customers from the data
# 03.01. Creating customer activity flags
# 03.02. Re-running previously ran queries using the refined data
# 03.02.01. Identifying the busiest day of the week
# 03.02.02. Identifying the busiest hour of the day
# 03.02.03. Identifying the hour of the day where the most money is spent
# 03.02.04. Identifying customer purchases by product price ranges
# 03.02.05. Identifying popular product types
# 03.02.06. Identifying the distribution customers by their total number of orders
# 03.02.07. Identifying the distribution customers by their order frequencies
# 03.02.08. Identifying the distribution of customers by regions
# 04. Customer profiling
# 04.01. Creating age and income based profiling variable
# 04.01.01. Create age buckets
# 04.01.02. Creating income buckets
# 04.01.03. Create profiling variable: Age_income_level
# 04.02 Create profiling variable: Age family status
# 04.03. Creating a profiling variable: Alcohol consumer vs. Non-alcohol consumer
# 04.04 Creating a profiling variable: Pet ownership by family status
# 04.05 Creating a profiling variable: Family status and number of dependants
# 04.06 Creating a profiling variable: Shopping day time
# 05. Exporting data

# 01. Importing libraries and data

In [4]:
# Import libraries

import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import scipy

In [5]:
# Defining the default path for accessing files

path = r'/Users/bladael/Documents/Learning/CareerFoundry_DA/Data Immersion/Achievement 4/06-2023 Instacart Basket Analysis'

In [6]:
# Import data

ords_prods_custs_merge = pd.read_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'orders_products_customers_merged.pkl'))

In [7]:
# Check the shape of the loaded dataframe

ords_prods_custs_merge.shape

(32404859, 33)

In [8]:
# Change the pandas display option to view the full list of columns

pd.options.display.max_columns = None

In [9]:
# Check the head of the loaded dataframe

ords_prods_custs_merge.head()

,order_id,user_id,order_sequence_number,order_days_of_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,prices,price_range_loc,busiest_day,Busiest_Days,busiest_period_of_day,max_order,loyalty_flag,total_ord_spend,total_ord_count,avg_ord_spend,spending_flag,median_frequency,frequency_flag,Gender,STATE,Age,date_joined,n_dependants,fam_status,income,_merge
0,2539329,1,1,2,8,0.0,196,1,0,Soda,77,7,9.0,Mid-range product,Regularly busy,Normal day,Average orders,10,New customer,375.7,10,37.57,High spender,20.0,Regular customer,Female,Alabama,31,2/17/2019,3,married,40423,both
1,2398795,1,2,3,7,15.0,196,1,1,Soda,77,7,9.0,Mid-range product,Regularly busy,Slowest day,Average orders,10,New customer,375.7,10,37.57,High spender,20.0,Regular customer,Female,Alabama,31,2/17/2019,3,married,40423,both
2,473747,1,3,3,12,21.0,196,1,1,Soda,77,7,9.0,Mid-range product,Regularly busy,Slowest day,Most orders,10,New customer,375.7,10,37.57,High spender,20.0,Regular customer,Female,Alabama,31,2/17/2019,3,married,40423,both
3,2254736,1,4,4,7,29.0,196,1,1,Soda,77,7,9.0,Mid-range product,Least busy,Slowest day,Average orders,10,New customer,375.7,10,37.57,High spender,20.0,Regular customer,Female,Alabama,31,2/17/2019,3,married,40423,both
4,431534,1,5,4,15,28.0,196,1,1,Soda,77,7,9.0,Mid-range product,Least busy,Slowest day,Most orders,10,New customer,375.7,10,37.57,High spender,20.0,Regular customer,Female,Alabama,31,2/17/2019,3,married,40423,both


In [10]:
# Noticed the '_merge' indicator column in the master dataframe that was just loaded. Delete this column.

del ords_prods_custs_merge['_merge']

In [11]:
# Check the head again

ords_prods_custs_merge.head()

,order_id,user_id,order_sequence_number,order_days_of_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,prices,price_range_loc,busiest_day,Busiest_Days,busiest_period_of_day,max_order,loyalty_flag,total_ord_spend,total_ord_count,avg_ord_spend,spending_flag,median_frequency,frequency_flag,Gender,STATE,Age,date_joined,n_dependants,fam_status,income
0,2539329,1,1,2,8,0.0,196,1,0,Soda,77,7,9.0,Mid-range product,Regularly busy,Normal day,Average orders,10,New customer,375.7,10,37.57,High spender,20.0,Regular customer,Female,Alabama,31,2/17/2019,3,married,40423
1,2398795,1,2,3,7,15.0,196,1,1,Soda,77,7,9.0,Mid-range product,Regularly busy,Slowest day,Average orders,10,New customer,375.7,10,37.57,High spender,20.0,Regular customer,Female,Alabama,31,2/17/2019,3,married,40423
2,473747,1,3,3,12,21.0,196,1,1,Soda,77,7,9.0,Mid-range product,Regularly busy,Slowest day,Most orders,10,New customer,375.7,10,37.57,High spender,20.0,Regular customer,Female,Alabama,31,2/17/2019,3,married,40423
3,2254736,1,4,4,7,29.0,196,1,1,Soda,77,7,9.0,Mid-range product,Least busy,Slowest day,Average orders,10,New customer,375.7,10,37.57,High spender,20.0,Regular customer,Female,Alabama,31,2/17/2019,3,married,40423
4,431534,1,5,4,15,28.0,196,1,1,Soda,77,7,9.0,Mid-range product,Least busy,Slowest day,Most orders,10,New customer,375.7,10,37.57,High spender,20.0,Regular customer,Female,Alabama,31,2/17/2019,3,married,40423


Task 2: CP: The PII implications were already addressed in the previous Exercise by not importing the customer first/last names

# 02. Analyzing the spending pattern by region

CP: Per the management request, the data needs to be analyzed by different geographic regions. A new dataframe for geographic areas will be created, then the initially loaded dataframe (the master dataframe) will be appended with a new column derivation. 

## 02.01. Creating Region column by merging

In [12]:
# Getting the list of the unique state names from the master data

ords_prods_custs_merge['STATE'].drop_duplicates()

0                     Alabama
59                    Indiana
131                 Louisiana
335                   Montana
540                  Oklahoma
839                 Tennessee
1153                     Iowa
1234                     Ohio
1391                 Virginia
1441              Connecticut
1510                 Kentucky
1517                   Oregon
1557             South Dakota
2226                 Michigan
3816               New Jersey
3855                  Arizona
3871                    Maine
3959                   Hawaii
4182                Minnesota
4210                 Illinois
5065                    Idaho
5084                   Kansas
5095           North Carolina
5140                  Vermont
5332                   Alaska
5908                 Delaware
5953                 Maryland
5986                 Nebraska
5997                   Nevada
6064                 Arkansas
6200                  Georgia
6278     District of Columbia
6736                     Utah
7013      

In [13]:
# Check the unique count of states in the master dataframe

ords_prods_custs_merge['STATE'].nunique()

51

In [14]:
# Region-state mapping data downloaded from the following source and saved in CSV format:
# https://simple.wikipedia.org/wiki/List_of_regions_of_the_United_States
# Loading it as a dataframe

df_region = pd.read_csv(os.path.join(path, '02 Data', 'Original Data', 'region_state.csv'), index_col = False)

In [15]:
# Check the loaded dafaframe

df_region

,Region,STATE
0,Northeast,Maine
1,Northeast,New Hampshire
2,Northeast,Vermont
3,Northeast,Massachusetts
4,Northeast,Rhode Island
5,Northeast,Connecticut
6,Northeast,New York
7,Northeast,Pennsylvania
8,Northeast,New Jersey
9,Midwest,Wisconsin


In [16]:
# Test the merge before creating a merged dataframe

pd.merge(ords_prods_custs_merge, df_region, on = 'STATE', how = 'outer', indicator = True)['_merge'].value_counts()

both          32404859
left_only            0
right_only           0
Name: _merge, dtype: int64

In [17]:
# Verified that the two dataframes have a full match
# Merge two dataframes using a left join

ords_prods_custs_merge = pd.merge(ords_prods_custs_merge, df_region, on = 'STATE', how = 'left', indicator = True)

In [18]:
# Check the head of the master dataframe after merging

ords_prods_custs_merge.head()

,order_id,user_id,order_sequence_number,order_days_of_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,prices,price_range_loc,busiest_day,Busiest_Days,busiest_period_of_day,max_order,loyalty_flag,total_ord_spend,total_ord_count,avg_ord_spend,spending_flag,median_frequency,frequency_flag,Gender,STATE,Age,date_joined,n_dependants,fam_status,income,Region,_merge
0,2539329,1,1,2,8,0.0,196,1,0,Soda,77,7,9.0,Mid-range product,Regularly busy,Normal day,Average orders,10,New customer,375.7,10,37.57,High spender,20.0,Regular customer,Female,Alabama,31,2/17/2019,3,married,40423,South,both
1,2398795,1,2,3,7,15.0,196,1,1,Soda,77,7,9.0,Mid-range product,Regularly busy,Slowest day,Average orders,10,New customer,375.7,10,37.57,High spender,20.0,Regular customer,Female,Alabama,31,2/17/2019,3,married,40423,South,both
2,473747,1,3,3,12,21.0,196,1,1,Soda,77,7,9.0,Mid-range product,Regularly busy,Slowest day,Most orders,10,New customer,375.7,10,37.57,High spender,20.0,Regular customer,Female,Alabama,31,2/17/2019,3,married,40423,South,both
3,2254736,1,4,4,7,29.0,196,1,1,Soda,77,7,9.0,Mid-range product,Least busy,Slowest day,Average orders,10,New customer,375.7,10,37.57,High spender,20.0,Regular customer,Female,Alabama,31,2/17/2019,3,married,40423,South,both
4,431534,1,5,4,15,28.0,196,1,1,Soda,77,7,9.0,Mid-range product,Least busy,Slowest day,Most orders,10,New customer,375.7,10,37.57,High spender,20.0,Regular customer,Female,Alabama,31,2/17/2019,3,married,40423,South,both


In [19]:
# Check the '_merge' column counts

ords_prods_custs_merge['_merge'].value_counts()

both          32404859
left_only            0
right_only           0
Name: _merge, dtype: int64

In [20]:
# Delete the '_merge' indicator column upon checking

del ords_prods_custs_merge['_merge']

In [21]:
# Delete 'df_region' after verifying that it was merged successfully to the master dataframe to save memory

del df_region

## 02.02. Grouping values (crosstab)

In [22]:
# To avoid double counting users, create a subset of users with region and spending flag columns and remove duplicates
# Then, use the subset to create a crosstab

region_spend_unique = ords_prods_custs_merge[['user_id', 'Region' , 'spending_flag']].drop_duplicates()
crosstab_region_spend = pd.crosstab(region_spend_unique['Region'], region_spend_unique['spending_flag'], dropna = False)

In [23]:
# Check the crosstab

crosstab_region_spend

spending_flag,High spender,Low spender
Region,,
Midwest,47945,574
Northeast,35942,446
South,67787,950
West,51848,717


In [24]:
# Check the number of the unique user_ids from the original master dataframe to cross check

ords_prods_custs_merge['user_id'].nunique(dropna = False)

206209

In [25]:
# Check the sum of the crosstab values to cross check

crosstab_region_spend['High spender'].sum() + crosstab_region_spend['Low spender'].sum()

206209

In [26]:
# Export the crosstab

crosstab_region_spend.to_csv(os.path.join(path, '04 Analysis', 'Reports', 'spend_by_region.csv'))

In [27]:
# Delete the crosstab related dataframes created upon exporting to save memory

del region_spend_unique
del crosstab_region_spend

# 03. Excluding low-activity customers from the data

CP: Per the request of CFO, creating the customer activity flags to filter the low-activity customers (customers with less than 5 orders total

## 03.01. Creating customer activity flags

In [28]:
# Check the head of the currently available columns

ords_prods_custs_merge.head()

,order_id,user_id,order_sequence_number,order_days_of_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,prices,price_range_loc,busiest_day,Busiest_Days,busiest_period_of_day,max_order,loyalty_flag,total_ord_spend,total_ord_count,avg_ord_spend,spending_flag,median_frequency,frequency_flag,Gender,STATE,Age,date_joined,n_dependants,fam_status,income,Region
0,2539329,1,1,2,8,0.0,196,1,0,Soda,77,7,9.0,Mid-range product,Regularly busy,Normal day,Average orders,10,New customer,375.7,10,37.57,High spender,20.0,Regular customer,Female,Alabama,31,2/17/2019,3,married,40423,South
1,2398795,1,2,3,7,15.0,196,1,1,Soda,77,7,9.0,Mid-range product,Regularly busy,Slowest day,Average orders,10,New customer,375.7,10,37.57,High spender,20.0,Regular customer,Female,Alabama,31,2/17/2019,3,married,40423,South
2,473747,1,3,3,12,21.0,196,1,1,Soda,77,7,9.0,Mid-range product,Regularly busy,Slowest day,Most orders,10,New customer,375.7,10,37.57,High spender,20.0,Regular customer,Female,Alabama,31,2/17/2019,3,married,40423,South
3,2254736,1,4,4,7,29.0,196,1,1,Soda,77,7,9.0,Mid-range product,Least busy,Slowest day,Average orders,10,New customer,375.7,10,37.57,High spender,20.0,Regular customer,Female,Alabama,31,2/17/2019,3,married,40423,South
4,431534,1,5,4,15,28.0,196,1,1,Soda,77,7,9.0,Mid-range product,Least busy,Slowest day,Most orders,10,New customer,375.7,10,37.57,High spender,20.0,Regular customer,Female,Alabama,31,2/17/2019,3,married,40423,South


In [29]:
# From the previous Exercise, a column with the total number of orders placed by user was created ('total_ord_count')
# Create a list of customr activity flags using the 'total_ord_count' column

cust_act = []

for value in ords_prods_custs_merge['total_ord_count']:
    if value < 5:
        cust_act.append("Low-activity")
    else:
        cust_act.append("Regular-activity")

In [30]:
# Check the list created

cust_act

['Regular-activity',
 'Regular-activity',
 'Regular-activity',
 'Regular-activity',
 'Regular-activity',
 'Regular-activity',
 'Regular-activity',
 'Regular-activity',
 'Regular-activity',
 'Regular-activity',
 'Regular-activity',
 'Regular-activity',
 'Regular-activity',
 'Regular-activity',
 'Regular-activity',
 'Regular-activity',
 'Regular-activity',
 'Regular-activity',
 'Regular-activity',
 'Regular-activity',
 'Regular-activity',
 'Regular-activity',
 'Regular-activity',
 'Regular-activity',
 'Regular-activity',
 'Regular-activity',
 'Regular-activity',
 'Regular-activity',
 'Regular-activity',
 'Regular-activity',
 'Regular-activity',
 'Regular-activity',
 'Regular-activity',
 'Regular-activity',
 'Regular-activity',
 'Regular-activity',
 'Regular-activity',
 'Regular-activity',
 'Regular-activity',
 'Regular-activity',
 'Regular-activity',
 'Regular-activity',
 'Regular-activity',
 'Regular-activity',
 'Regular-activity',
 'Regular-activity',
 'Regular-activity',
 'Regular-act

In [31]:
# Derive 'customer_activity' flag column based on the 'cust_act' list created above

ords_prods_custs_merge['customer_activity'] = cust_act

In [32]:
# Check the head of the revised master dataframe

ords_prods_custs_merge.head()

,order_id,user_id,order_sequence_number,order_days_of_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,prices,price_range_loc,busiest_day,Busiest_Days,busiest_period_of_day,max_order,loyalty_flag,total_ord_spend,total_ord_count,avg_ord_spend,spending_flag,median_frequency,frequency_flag,Gender,STATE,Age,date_joined,n_dependants,fam_status,income,Region,customer_activity
0,2539329,1,1,2,8,0.0,196,1,0,Soda,77,7,9.0,Mid-range product,Regularly busy,Normal day,Average orders,10,New customer,375.7,10,37.57,High spender,20.0,Regular customer,Female,Alabama,31,2/17/2019,3,married,40423,South,Regular-activity
1,2398795,1,2,3,7,15.0,196,1,1,Soda,77,7,9.0,Mid-range product,Regularly busy,Slowest day,Average orders,10,New customer,375.7,10,37.57,High spender,20.0,Regular customer,Female,Alabama,31,2/17/2019,3,married,40423,South,Regular-activity
2,473747,1,3,3,12,21.0,196,1,1,Soda,77,7,9.0,Mid-range product,Regularly busy,Slowest day,Most orders,10,New customer,375.7,10,37.57,High spender,20.0,Regular customer,Female,Alabama,31,2/17/2019,3,married,40423,South,Regular-activity
3,2254736,1,4,4,7,29.0,196,1,1,Soda,77,7,9.0,Mid-range product,Least busy,Slowest day,Average orders,10,New customer,375.7,10,37.57,High spender,20.0,Regular customer,Female,Alabama,31,2/17/2019,3,married,40423,South,Regular-activity
4,431534,1,5,4,15,28.0,196,1,1,Soda,77,7,9.0,Mid-range product,Least busy,Slowest day,Most orders,10,New customer,375.7,10,37.57,High spender,20.0,Regular customer,Female,Alabama,31,2/17/2019,3,married,40423,South,Regular-activity


In [33]:
# Delete the list 'cust_act' upon verifying its merge to the master dataframe to save memory

del cust_act

In [34]:
# Create a refined dataframe excluding low-activity customers

ords_prods_reg_custs_merge = ords_prods_custs_merge[ords_prods_custs_merge['customer_activity'] == 'Regular-activity']

In [35]:
# Check the data shape of the new dataframe excluding the low-activitiy customer related data

ords_prods_reg_custs_merge.shape

(30964518, 34)

In [36]:
# Check the head of the new dataframe excluding the low-activitiy customer related data

ords_prods_reg_custs_merge.head()

,order_id,user_id,order_sequence_number,order_days_of_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,prices,price_range_loc,busiest_day,Busiest_Days,busiest_period_of_day,max_order,loyalty_flag,total_ord_spend,total_ord_count,avg_ord_spend,spending_flag,median_frequency,frequency_flag,Gender,STATE,Age,date_joined,n_dependants,fam_status,income,Region,customer_activity
0,2539329,1,1,2,8,0.0,196,1,0,Soda,77,7,9.0,Mid-range product,Regularly busy,Normal day,Average orders,10,New customer,375.7,10,37.57,High spender,20.0,Regular customer,Female,Alabama,31,2/17/2019,3,married,40423,South,Regular-activity
1,2398795,1,2,3,7,15.0,196,1,1,Soda,77,7,9.0,Mid-range product,Regularly busy,Slowest day,Average orders,10,New customer,375.7,10,37.57,High spender,20.0,Regular customer,Female,Alabama,31,2/17/2019,3,married,40423,South,Regular-activity
2,473747,1,3,3,12,21.0,196,1,1,Soda,77,7,9.0,Mid-range product,Regularly busy,Slowest day,Most orders,10,New customer,375.7,10,37.57,High spender,20.0,Regular customer,Female,Alabama,31,2/17/2019,3,married,40423,South,Regular-activity
3,2254736,1,4,4,7,29.0,196,1,1,Soda,77,7,9.0,Mid-range product,Least busy,Slowest day,Average orders,10,New customer,375.7,10,37.57,High spender,20.0,Regular customer,Female,Alabama,31,2/17/2019,3,married,40423,South,Regular-activity
4,431534,1,5,4,15,28.0,196,1,1,Soda,77,7,9.0,Mid-range product,Least busy,Slowest day,Most orders,10,New customer,375.7,10,37.57,High spender,20.0,Regular customer,Female,Alabama,31,2/17/2019,3,married,40423,South,Regular-activity


In [37]:
# Check the customer activity flag counts of the master dataframe for a comparison

ords_prods_custs_merge['customer_activity'].value_counts()

Regular-activity    30964518
Low-activity         1440341
Name: customer_activity, dtype: int64

In [38]:
# Check the unique count of user ids after applying the exclusion flag

ords_prods_reg_custs_merge['user_id'].nunique()

162627

In [39]:
# Check the unique count of user ids after applying the exclusion flag

ords_prods_reg_custs_merge['order_id'].nunique()

3064361

In [40]:
# Export the low-activity customer transactions 

ords_prods_custs_merge[ords_prods_custs_merge['customer_activity'] == 'Low-activity'].to_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'orders_low_activity_customers.pkl'))

In [41]:
# Given that now the new dataframe 'ords_prods_reg_custs_merge' will be used,
# Delete the old dataframe 'ords_prods_custs_merge' to avoid memory leakage

del ords_prods_custs_merge

## 03.02. Re-running previously ran queries using the refined data

CP: There were certain business questions that have been already answered in the previous Exercises including the low-activity level customers. Given that CFO want to exclude the low-activity level customers from analyses, the analyses need to be ran again excluding them 

### 03.02.01. Identifying the busiest day of the week

In [42]:
# Check the unique order counts based on order_days_of_week

ords_prods_reg_custs_merge.groupby(['order_days_of_week'])['order_id'].nunique().sort_values(ascending = False)

order_days_of_week
1    531257
0    529221
2    421534
5    407454
6    397930
3    393437
4    383528
Name: order_id, dtype: int64

CP: Two busiest days: 0 & 1 (Saturday & Sunday). Two slowest days: 3 & 4 (Tuesday & Wednesday)

In [43]:
# Create a dataframe using the value count results above for descriptive stats to identify basis for bucketing

df_ord_day = ords_prods_reg_custs_merge.groupby(['order_days_of_week'])['order_id'].nunique().sort_values(ascending=False).rename_axis('day (0=Sat, 6=Fri)').reset_index(name='unique_order_counts')

In [44]:
# Export the result

df_ord_day.to_csv(os.path.join(path, '04 Analysis', 'Reports', 'busiest_days_refined.csv'), index = False)

In [45]:
# Delete the dataframe 'df_ord_day' to save memory

del df_ord_day

### 03.02.02. Identifying the busiest hour of the day

In [46]:
# Check the unique order counts based on order_hour_of_day

ords_prods_reg_custs_merge.groupby(['order_hour_of_day'])['order_id'].nunique().sort_values(ascending = False)

order_hour_of_day
10    260111
11    255569
15    252989
14    252847
13    248712
12    243654
16    243327
9     233810
17    203000
8     161957
18    161929
19    124715
20     93341
7      83633
21     70100
22     54834
23     35817
6      27753
0      20291
1      10975
5       8554
2       6684
4       4909
3       4850
Name: order_id, dtype: int64

CP: Two busiest hours: 11 AM & 12 PM. Two slowest hours: 5 AM & 4 AM

In [47]:
# Create a dataframe using the value count results above for descriptive stats to identify basis for bucketing

df_ord_hr = ords_prods_reg_custs_merge.groupby(['order_hour_of_day'])['order_id'].nunique().sort_values(ascending = False).rename_axis('hour (0=1 AM, 23=12 AM)').reset_index(name='unique_order_counts')

In [48]:
# Export the result

df_ord_hr.to_csv(os.path.join(path, '04 Analysis', 'Reports', 'busiest_hours_refined.csv'), index = False)

In [49]:
# Delete the dataframe 'df_ord_hr' to save memory

del df_ord_hr

### 03.02.03. Identifying the hour of the day where the most money is spent

In [50]:
# Use groupby and agg function to calculate the hours where the most money was spent by customers

ords_prods_reg_custs_merge.groupby(['order_hour_of_day']).agg({'prices': ['sum']}).reset_index()

,order_hour_of_day,prices
,,sum
0,0,1625007.5
1,1,855655.6
2,2,512533.9
3,3,381978.4
4,4,398014.7
5,5,660968.6
6,6,2197137.1
7,7,6750538.7
8,8,12922482.5


CP: The most money was spent at 11 AM

In [51]:
# Create a dataframe using the value count results for reporting

money_hr = ords_prods_reg_custs_merge.groupby(['order_hour_of_day']).agg({'prices': ['sum']}).reset_index()

In [52]:
# Export the result

money_hr.to_csv(os.path.join(path, '04 Analysis', 'Reports', 'most_money_spent_hours_refined.csv'), index = False)

In [53]:
# Delete the dataframe 'money_hr' to save memory

del money_hr

### 03.02.04. Identifying customer purchases by product price ranges

In [54]:
# Check the value counts of 'price_range_loc'

ords_prods_reg_custs_merge['price_range_loc'].value_counts(dropna = False)

Mid-range product     20891735
Low-range product      9674830
High-range product      393076
Price unavailable         4877
Name: price_range_loc, dtype: int64

In [55]:
# Create a dataframe using the value count results above for reporting

df_price_range_counts = ords_prods_reg_custs_merge['price_range_loc'].value_counts(dropna = False).rename_axis('price_range').reset_index(name='order_counts')

In [56]:
# Export the result

df_price_range_counts.to_csv(os.path.join(path, '04 Analysis', 'Reports', 'order_counts_by_price_range_refined.csv'), index = False)

In [57]:
# Delete the dataframe 'df_price_range_counts' to save memory

del df_price_range_counts

### 03.02.05. Identifying popular product types

In [58]:
# Obtain the count of orders per each department and create a dataframe

df_dep_ord = ords_prods_reg_custs_merge['department_id'].value_counts(dropna = False).rename_axis('department_id').reset_index(name='order_counts')

In [59]:
# Derive a new column for the average number of orders per each deparment by a given user
# by dividing the order counts by the total number of unique users

df_dep_ord['avg_order_per_user'] = (df_dep_ord['order_counts'] / ords_prods_reg_custs_merge['user_id'].nunique())

In [60]:
# Import department details to display the department name, not just department_id in the master dataframe

df_depts = pd.read_csv(os.path.join(path, '02 Data', 'Original Data', 'departments.csv'), index_col = False).T.reset_index()

In [61]:
# Check the imported dataframe

df_depts

,index,0
0,department_id,department
1,1,frozen
2,2,other
3,3,bakery
4,4,produce
5,5,alcohol
6,6,international
7,7,beverages
8,8,pets
9,9,dry goods pasta


In [62]:
# Take the correct array for the column heading

new_head = df_depts.iloc[0]

In [63]:
# Take on values from row 1 and onwards for the dataframe
df_depts = df_depts[1:]

In [64]:
# Assign the new head

df_depts.columns = new_head

In [65]:
# Check the revised dataframe

df_depts

,department_id,department
1,1,frozen
2,2,other
3,3,bakery
4,4,produce
5,5,alcohol
6,6,international
7,7,beverages
8,8,pets
9,9,dry goods pasta
10,10,bulk


In [66]:
# Check the information of the dataframe to be merged

df_depts.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21 entries, 1 to 21
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   department_id  21 non-null     object
 1   department     21 non-null     object
dtypes: object(2)
memory usage: 468.0+ bytes


In [67]:
# Check the information of the dataframe to be merged

df_dep_ord.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21 entries, 0 to 20
Data columns (total 3 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   department_id       21 non-null     int64  
 1   order_counts        21 non-null     int64  
 2   avg_order_per_user  21 non-null     float64
dtypes: float64(1), int64(2)
memory usage: 632.0 bytes


In [68]:
# Change the 'department_id' field data type in 'df_depts' into int

df_depts['department_id'] = df_depts['department_id'].astype('int')

/var/folders/sd/bl3lrkt53x14smrncn8mblp00000gn/T/ipykernel_1296/1828109180.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_depts['department_id'] = df_depts['department_id'].astype('int')


In [69]:
# Check the information of 'df_depts' dataframe to check the data types again

df_depts.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21 entries, 1 to 21
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   department_id  21 non-null     int64 
 1   department     21 non-null     object
dtypes: int64(1), object(1)
memory usage: 468.0+ bytes


In [70]:
# Add the department name column to the master dataframe

df_dep_ord = pd.merge(df_dep_ord, df_depts, on = 'department_id', how = 'left', indicator = False)

In [71]:
# Display the results

df_dep_ord

,department_id,order_counts,avg_order_per_user,department
0,4,9079267,55.828780,produce
1,16,5177180,31.834689,dairy eggs
2,19,2766402,17.010718,snacks
3,7,2571897,15.814699,beverages
4,1,2121725,13.046573,frozen
5,13,1782705,10.961925,pantry
6,3,1120824,6.891992,bakery
7,15,1012072,6.223272,canned goods
8,20,1003831,6.172597,deli
9,9,822136,5.055348,dry goods pasta


CP: Based on the analysis, the items from the 'produce' department was the most popular products. On an average, an Instacart customer ordered 55.8 products from the 'produce' department during their Instacart order histories.

In [72]:
# Export the result

df_dep_ord.to_csv(os.path.join(path, '04 Analysis', 'Reports', 'popular_department_refined.csv'), index = False)

In [73]:
# # Delete the list 'new_head' and dataframes 'df_depts' and 'df_dep_ord' to save memory

del new_head
del df_depts
del df_dep_ord

### 03.02.06. Identifying the distribution customers by their total number of orders

In [74]:
# Getting the number of unique user ids by different loyalty flags

ords_prods_reg_custs_merge.groupby(['loyalty_flag'])['user_id'].nunique().sort_values(ascending = False)

loyalty_flag
Regular customer    76864
New customer        68746
Loyal customer      17017
Name: user_id, dtype: int64

In [75]:
# Create a dataframe of the result for reporting

loyalty_summ = ords_prods_reg_custs_merge.groupby(['loyalty_flag'])['user_id'].nunique().sort_values(ascending = False).rename_axis('loyalty_flag').reset_index(name='unique_user_counts')

In [76]:
# Export the result

loyalty_summ.to_csv(os.path.join(path, '04 Analysis', 'Reports', 'customer_loyalty_summary.csv'), index = False)

In [77]:
# Delete 'loyalty_summ' to save memory

del loyalty_summ

### 03.02.07. Identifying the distribution customers by their order frequencies

In [78]:
# Getting the number of unique user ids by different frequency flags

ords_prods_reg_custs_merge.groupby(['frequency_flag'])['user_id'].nunique().sort_values(ascending = False)

frequency_flag
Frequent customer        85272
Regular customer         49403
Non-frequent customer    27952
Name: user_id, dtype: int64

In [79]:
# Create a dataframe of the result for reporting

frequency_summ = ords_prods_reg_custs_merge.groupby(['frequency_flag'])['user_id'].nunique().sort_values(ascending = False).rename_axis('loyalty_flag').reset_index(name='unique_user_counts')

In [80]:
# Export the result

frequency_summ.to_csv(os.path.join(path, '04 Analysis', 'Reports','customer_frequency_summary.csv'), index = False)

In [81]:
# Delete 'frequency_summ' to save memory

del frequency_summ

### 03.02.08. Identifying the distribution of customers by regions

In [82]:
# To avoid double counting users, create a subset of users with region and spending flag columns and remove duplicates
# Then, use the subset to create a crosstab

region_spend_unique_ref = ords_prods_reg_custs_merge[['user_id', 'Region' , 'spending_flag']].drop_duplicates()
crosstab_region_spend_ref = pd.crosstab(region_spend_unique_ref['Region'], region_spend_unique_ref['spending_flag'], dropna = False)

In [83]:
# Check the crosstab

crosstab_region_spend_ref

spending_flag,High spender,Low spender
Region,,
Midwest,38012,363
Northeast,28309,271
South,53624,567
West,41047,434


In [84]:
# Check the number of the unique user_ids from the original master dataframe to cross check

ords_prods_reg_custs_merge['user_id'].nunique(dropna = False)

162627

In [85]:
# Check the sum of the crosstab values to cross check

crosstab_region_spend_ref['High spender'].sum() + crosstab_region_spend_ref['Low spender'].sum()

162627

In [86]:
# Export the crosstab

crosstab_region_spend_ref.to_csv(os.path.join(path, '04 Analysis', 'Reports', 'spend_by_region_refined.csv'))

In [87]:
# Delete the crosstab related dataframes created upon exporting to save memory

del region_spend_unique_ref
del crosstab_region_spend_ref

# 04. Customer profiling

CP: The marketing and business strategy units at instacart want to create more-relevant marketing strategies. This section is dedicated to creating different customer profiling variables

## 04.01. Creating age and income based profiling variable

### 04.01.01. Create age buckets

In [88]:
# Analyze the descriptive stats over the age variable

ords_prods_reg_custs_merge['Age'].describe()

count    3.096452e+07
mean     4.946802e+01
std      1.848528e+01
min      1.800000e+01
25%      3.300000e+01
50%      4.900000e+01
75%      6.500000e+01
max      8.100000e+01
Name: Age, dtype: float64

CP: Based on the stats above, made a decision to use one standard deviation to create three age buckets: (1) mean - 1 std dev = Young adult, (2) mean + 1 std dev = Senior, (3) everyone else = Middle-aged

In [89]:
# Label Young Adults in a newly derived column of 'age_bucket'

ords_prods_reg_custs_merge.loc[ords_prods_reg_custs_merge['Age'] < (4.946802e+01 - 1.848528e+01), 'age_bucket'] = 'Young adult'

In [90]:
# Label Seniors in a newly derived column of 'age_bucket'

ords_prods_reg_custs_merge.loc[ords_prods_reg_custs_merge['Age'] > (4.946802e+01 + 1.848528e+01), 'age_bucket'] = 'Senior'

In [91]:
# Label Adults in a newly derived column of 'age_bucket'

ords_prods_reg_custs_merge.loc[(ords_prods_reg_custs_merge['Age'] >= (4.946802e+01 - 1.848528e+01)) & (ords_prods_reg_custs_merge['Age'] <= (4.946802e+01 + 1.848528e+01)), 'age_bucket'] = 'Middle-aged'

In [92]:
# Check the value counts of the 'age_bucket' column

ords_prods_reg_custs_merge['age_bucket'].value_counts(dropna = False)

Middle-aged    17856600
Senior          6803688
Young adult     6304230
Name: age_bucket, dtype: int64

### 04.01.02. Creating income buckets

In [93]:
# Analyze the descriptive stats over the income variable

ords_prods_reg_custs_merge['income'].describe()

count    3.096452e+07
mean     9.967582e+04
std      4.314184e+04
min      2.590300e+04
25%      6.729200e+04
50%      9.676500e+04
75%      1.281020e+05
max      5.939010e+05
Name: income, dtype: float64

CP: Based on the stats above, made a decision to use one standard deviation to create three age buckets: (1) mean - 1 std dev = Low income, (2) mean + 1 std dev = High income, (3) everything else = Middle income

In [94]:
# Create income flags in a new column 'income_bucket'

ords_prods_reg_custs_merge.loc[ords_prods_reg_custs_merge['income'] < (9.967582e+04 - 4.314184e+04), 'income_bucket'] = 'Low income'

In [95]:
# Create income flags in a new column 'income_bucket'

ords_prods_reg_custs_merge.loc[ords_prods_reg_custs_merge['income'] > (9.967582e+04 + 4.314184e+04), 'income_bucket'] = 'High income'

In [96]:
# Create income flags in a new column 'income_bucket'

ords_prods_reg_custs_merge.loc[(ords_prods_reg_custs_merge['income'] >= (9.967582e+04 - 4.314184e+04)) & (ords_prods_reg_custs_merge['income'] <= (9.967582e+04 + 4.314184e+04)), 'income_bucket'] = 'Middle income'

In [97]:
# Check the value counts of the 'income_bucket' column

ords_prods_reg_custs_merge['income_bucket'].value_counts(dropna = False)

Middle income    20923951
High income       5173055
Low income        4867512
Name: income_bucket, dtype: int64

### 04.01.03. Create profiling variable: Age_income_level 

In [98]:
# Create a new column by combining the variables from the two new columns created above

ords_prods_reg_custs_merge['Age_income_level'] = ords_prods_reg_custs_merge['age_bucket'].astype('str')+' '+ords_prods_reg_custs_merge['income_bucket'].astype('str')

In [99]:
# Check the head of the master dataframe to verify the column creation

ords_prods_reg_custs_merge.head()

,order_id,user_id,order_sequence_number,order_days_of_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,prices,price_range_loc,busiest_day,Busiest_Days,busiest_period_of_day,max_order,loyalty_flag,total_ord_spend,total_ord_count,avg_ord_spend,spending_flag,median_frequency,frequency_flag,Gender,STATE,Age,date_joined,n_dependants,fam_status,income,Region,customer_activity,age_bucket,income_bucket,Age_income_level
0,2539329,1,1,2,8,0.0,196,1,0,Soda,77,7,9.0,Mid-range product,Regularly busy,Normal day,Average orders,10,New customer,375.7,10,37.57,High spender,20.0,Regular customer,Female,Alabama,31,2/17/2019,3,married,40423,South,Regular-activity,Middle-aged,Low income,Middle-aged Low income
1,2398795,1,2,3,7,15.0,196,1,1,Soda,77,7,9.0,Mid-range product,Regularly busy,Slowest day,Average orders,10,New customer,375.7,10,37.57,High spender,20.0,Regular customer,Female,Alabama,31,2/17/2019,3,married,40423,South,Regular-activity,Middle-aged,Low income,Middle-aged Low income
2,473747,1,3,3,12,21.0,196,1,1,Soda,77,7,9.0,Mid-range product,Regularly busy,Slowest day,Most orders,10,New customer,375.7,10,37.57,High spender,20.0,Regular customer,Female,Alabama,31,2/17/2019,3,married,40423,South,Regular-activity,Middle-aged,Low income,Middle-aged Low income
3,2254736,1,4,4,7,29.0,196,1,1,Soda,77,7,9.0,Mid-range product,Least busy,Slowest day,Average orders,10,New customer,375.7,10,37.57,High spender,20.0,Regular customer,Female,Alabama,31,2/17/2019,3,married,40423,South,Regular-activity,Middle-aged,Low income,Middle-aged Low income
4,431534,1,5,4,15,28.0,196,1,1,Soda,77,7,9.0,Mid-range product,Least busy,Slowest day,Most orders,10,New customer,375.7,10,37.57,High spender,20.0,Regular customer,Female,Alabama,31,2/17/2019,3,married,40423,South,Regular-activity,Middle-aged,Low income,Middle-aged Low income


In [100]:
# Check the value counts of the profile variable column created

ords_prods_reg_custs_merge.groupby(['Age_income_level'])['user_id'].nunique()

Age_income_level
Middle-aged High income      16493
Middle-aged Low income       18256
Middle-aged Middle income    59097
Senior High income            8561
Senior Low income             5376
Senior Middle income         21839
Young adult High income        230
Young adult Low income       10033
Young adult Middle income    22742
Name: user_id, dtype: int64

## 04.02 Create profiling variable: Age family status

In [101]:
# Create another profiling variable using age bucket and family status

# Review one of the columns to be combined

ords_prods_reg_custs_merge['fam_status'].value_counts()

married                             21743700
single                               5094375
divorced/widowed                     2645271
living with parents and siblings     1481172
Name: fam_status, dtype: int64

In [102]:
# Combine the variables of the two columns to create a new profiling variable

ords_prods_reg_custs_merge['Age_fam_status'] = ords_prods_reg_custs_merge['age_bucket'].astype('str')+' '+ords_prods_reg_custs_merge['fam_status'].astype('str')

In [103]:
# Check the head of the master dataframe to verify the column creation

ords_prods_reg_custs_merge.head()

,order_id,user_id,order_sequence_number,order_days_of_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,prices,price_range_loc,busiest_day,Busiest_Days,busiest_period_of_day,max_order,loyalty_flag,total_ord_spend,total_ord_count,avg_ord_spend,spending_flag,median_frequency,frequency_flag,Gender,STATE,Age,date_joined,n_dependants,fam_status,income,Region,customer_activity,age_bucket,income_bucket,Age_income_level,Age_fam_status
0,2539329,1,1,2,8,0.0,196,1,0,Soda,77,7,9.0,Mid-range product,Regularly busy,Normal day,Average orders,10,New customer,375.7,10,37.57,High spender,20.0,Regular customer,Female,Alabama,31,2/17/2019,3,married,40423,South,Regular-activity,Middle-aged,Low income,Middle-aged Low income,Middle-aged married
1,2398795,1,2,3,7,15.0,196,1,1,Soda,77,7,9.0,Mid-range product,Regularly busy,Slowest day,Average orders,10,New customer,375.7,10,37.57,High spender,20.0,Regular customer,Female,Alabama,31,2/17/2019,3,married,40423,South,Regular-activity,Middle-aged,Low income,Middle-aged Low income,Middle-aged married
2,473747,1,3,3,12,21.0,196,1,1,Soda,77,7,9.0,Mid-range product,Regularly busy,Slowest day,Most orders,10,New customer,375.7,10,37.57,High spender,20.0,Regular customer,Female,Alabama,31,2/17/2019,3,married,40423,South,Regular-activity,Middle-aged,Low income,Middle-aged Low income,Middle-aged married
3,2254736,1,4,4,7,29.0,196,1,1,Soda,77,7,9.0,Mid-range product,Least busy,Slowest day,Average orders,10,New customer,375.7,10,37.57,High spender,20.0,Regular customer,Female,Alabama,31,2/17/2019,3,married,40423,South,Regular-activity,Middle-aged,Low income,Middle-aged Low income,Middle-aged married
4,431534,1,5,4,15,28.0,196,1,1,Soda,77,7,9.0,Mid-range product,Least busy,Slowest day,Most orders,10,New customer,375.7,10,37.57,High spender,20.0,Regular customer,Female,Alabama,31,2/17/2019,3,married,40423,South,Regular-activity,Middle-aged,Low income,Middle-aged Low income,Middle-aged married


In [104]:
# Check the value counts of the profile variable column created

ords_prods_reg_custs_merge.groupby(['Age_fam_status'])['user_id'].nunique()

Age_fam_status
Middle-aged divorced/widowed                     5007
Middle-aged married                             70307
Middle-aged single                              18532
Senior divorced/widowed                          8824
Senior married                                  26952
Young adult living with parents and siblings     7608
Young adult married                             17035
Young adult single                               8362
Name: user_id, dtype: int64

## 04.03. Creating a profiling variable: Alcohol consumer vs. Non-alcohol consumer

CP: This profiling variable is to group and distinguish between alcohol consumers and non-alcohol consumers by age buckets

In [105]:
# Identify the customers with alcohol purchases vs not by creating a subset
# Create a subset to only include user id , order id, and department id
# Filter the subset by the rows containing only alcohol
# Remove the duplicated user ids from the filtered subset and assign value 'Alcohol consumer' to a new column

subset_dept_cat = ords_prods_reg_custs_merge[['user_id','order_id','department_id']]
user_alcohol = subset_dept_cat[subset_dept_cat['department_id'] == 5]
user_alcohol_no_dup = user_alcohol[['user_id']].drop_duplicates()
user_alcohol_no_dup['Alcohol_flag'] = 'Alcohol consumer'

In [106]:
# Check the shape of the alcohol purchaser dataframe

user_alcohol_no_dup.shape

(13150, 2)

In [107]:
# Check the head of the alcohol purchase dataframe

user_alcohol_no_dup.head()

,user_id,Alcohol_flag
377,21,Alcohol consumer
568,31,Alcohol consumer
1412,98,Alcohol consumer
1488,109,Alcohol consumer
1523,185,Alcohol consumer


In [108]:
# Check one sample user to see whether the person actually purchased alcohol from the master dataframe

ords_prods_reg_custs_merge[(ords_prods_reg_custs_merge['user_id'] == 21) & (ords_prods_reg_custs_merge['department_id'] == 5)]

,order_id,user_id,order_sequence_number,order_days_of_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,prices,price_range_loc,busiest_day,Busiest_Days,busiest_period_of_day,max_order,loyalty_flag,total_ord_spend,total_ord_count,avg_ord_spend,spending_flag,median_frequency,frequency_flag,Gender,STATE,Age,date_joined,n_dependants,fam_status,income,Region,customer_activity,age_bucket,income_bucket,Age_income_level,Age_fam_status
377,3288630,21,8,2,13,18.0,38444,2,0,Chardonnay,62,5,6.8,Mid-range product,Regularly busy,Normal day,Most orders,33,Regular customer,1437.6,33,43.563636,High spender,7.0,Frequent customer,Male,Montana,71,6/18/2019,0,divorced/widowed,124643,West,Regular-activity,Senior,Middle income,Senior Middle income,Senior divorced/widowed
453,62373,21,5,1,14,7.0,41131,2,0,India Pale Ale,27,5,13.9,Mid-range product,Regularly busy,Busiest day,Most orders,33,Regular customer,1437.6,33,43.563636,High spender,7.0,Frequent customer,Male,Montana,71,6/18/2019,0,divorced/widowed,124643,West,Regular-activity,Senior,Middle income,Senior Middle income,Senior divorced/widowed
454,62373,21,5,1,14,7.0,12129,3,0,12 Oz Lager,27,5,13.2,Mid-range product,Regularly busy,Busiest day,Most orders,33,Regular customer,1437.6,33,43.563636,High spender,7.0,Frequent customer,Male,Montana,71,6/18/2019,0,divorced/widowed,124643,West,Regular-activity,Senior,Middle income,Senior Middle income,Senior divorced/widowed
455,62373,21,5,1,14,7.0,14708,8,0,Stoli Blueberi,124,5,8.1,Mid-range product,Regularly busy,Busiest day,Most orders,33,Regular customer,1437.6,33,43.563636,High spender,7.0,Frequent customer,Male,Montana,71,6/18/2019,0,divorced/widowed,124643,West,Regular-activity,Senior,Middle income,Senior Middle income,Senior divorced/widowed
467,3288630,21,8,2,13,18.0,20272,1,0,Morning Fog Chardonnay,62,5,6.1,Mid-range product,Regularly busy,Normal day,Most orders,33,Regular customer,1437.6,33,43.563636,High spender,7.0,Frequent customer,Male,Montana,71,6/18/2019,0,divorced/widowed,124643,West,Regular-activity,Senior,Middle income,Senior Middle income,Senior divorced/widowed
468,3288630,21,8,2,13,18.0,12013,3,0,Pinot Noir,28,5,8.9,Mid-range product,Regularly busy,Normal day,Most orders,33,Regular customer,1437.6,33,43.563636,High spender,7.0,Frequent customer,Male,Montana,71,6/18/2019,0,divorced/widowed,124643,West,Regular-activity,Senior,Middle income,Senior Middle income,Senior divorced/widowed
469,3288630,21,8,2,13,18.0,16139,4,0,Clara,27,5,15.0,Mid-range product,Regularly busy,Normal day,Most orders,33,Regular customer,1437.6,33,43.563636,High spender,7.0,Frequent customer,Male,Montana,71,6/18/2019,0,divorced/widowed,124643,West,Regular-activity,Senior,Middle income,Senior Middle income,Senior divorced/widowed
475,1573906,21,10,3,10,6.0,33065,3,0,Cabernet Sauvignon,28,5,7.5,Mid-range product,Regularly busy,Slowest day,Most orders,33,Regular customer,1437.6,33,43.563636,High spender,7.0,Frequent customer,Male,Montana,71,6/18/2019,0,divorced/widowed,124643,West,Regular-activity,Senior,Middle income,Senior Middle income,Senior divorced/widowed
476,1573906,21,10,3,10,6.0,15720,4,0,Premium Beer,27,5,5.5,Mid-range product,Regularly busy,Slowest day,Most orders,33,Regular customer,1437.6,33,43.563636,High spender,7.0,Frequent customer,Male,Montana,71,6/18/2019,0,divorced/widowed,124643,West,Regular-activity,Senior,Middle income,Senior Middle income,Senior divorced/widowed
477,1573906,21,10,3,10,6.0,23504,5,0,Vintner's Reserve Chardonnay,62,5,3.0,Low-range product,Regularly busy,Slowest day,Most orders,33,Regular customer,1437.6,33,43.563636,High spender,7.0,Frequent customer,Male,Montana,71,6/18/2019,0,divorced/widowed,124643,West,Regular-activity,Senior,Middle income,Senior Middle income,Senior divorced/widowed


In [109]:
# Merge the filtered subset of alcohol consumers to the master dataframe using merge function

ords_prods_reg_custs_merge = pd.merge(ords_prods_reg_custs_merge, user_alcohol_no_dup, on = 'user_id', how = 'left', indicator = False)

In [110]:
# Check the value counts of the new column added to the master dataframe

ords_prods_reg_custs_merge['Alcohol_flag'].value_counts(dropna = False)

NaN                 28634030
Alcohol consumer     2330488
Name: Alcohol_flag, dtype: int64

In [111]:
# Also check whether the number of the unique alcohol consumers equal to the one from the filtered subset

ords_prods_reg_custs_merge.groupby('Alcohol_flag')['user_id'].nunique()

Alcohol_flag
Alcohol consumer    13150
Name: user_id, dtype: int64

In [112]:
# For the unmatched rows in the alcohol_flag column, assign value of Non-alcohol consumer using loc function

ords_prods_reg_custs_merge.loc[ords_prods_reg_custs_merge['Alcohol_flag'].isnull(), 'Alcohol_flag'] = 'Non-alcohol consumer'

In [113]:
# Check the value counts again

ords_prods_reg_custs_merge['Alcohol_flag'].value_counts(dropna = False)

Non-alcohol consumer    28634030
Alcohol consumer         2330488
Name: Alcohol_flag, dtype: int64

In [114]:
# Delete the subset dataframes after verifying that it was successfully merged to save memory

del user_alcohol
del user_alcohol_no_dup

In [115]:
# Concatenate 'age_bucket' and 'Alcohol_flag' to create a new profile variable

ords_prods_reg_custs_merge['Age_alcohol_level'] = ords_prods_reg_custs_merge['age_bucket']+' '+ords_prods_reg_custs_merge['Alcohol_flag']

In [116]:
# Change the data type of the concatenated field into str

ords_prods_reg_custs_merge['Age_alcohol_level'] = ords_prods_reg_custs_merge['Age_alcohol_level'].astype('str')

In [117]:
# Check the shape of the appended dataframe

ords_prods_reg_custs_merge.shape

(30964518, 40)

In [118]:
# Check the head of the appended dataframe

ords_prods_reg_custs_merge.head()

,order_id,user_id,order_sequence_number,order_days_of_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,prices,price_range_loc,busiest_day,Busiest_Days,busiest_period_of_day,max_order,loyalty_flag,total_ord_spend,total_ord_count,avg_ord_spend,spending_flag,median_frequency,frequency_flag,Gender,STATE,Age,date_joined,n_dependants,fam_status,income,Region,customer_activity,age_bucket,income_bucket,Age_income_level,Age_fam_status,Alcohol_flag,Age_alcohol_level
0,2539329,1,1,2,8,0.0,196,1,0,Soda,77,7,9.0,Mid-range product,Regularly busy,Normal day,Average orders,10,New customer,375.7,10,37.57,High spender,20.0,Regular customer,Female,Alabama,31,2/17/2019,3,married,40423,South,Regular-activity,Middle-aged,Low income,Middle-aged Low income,Middle-aged married,Non-alcohol consumer,Middle-aged Non-alcohol consumer
1,2398795,1,2,3,7,15.0,196,1,1,Soda,77,7,9.0,Mid-range product,Regularly busy,Slowest day,Average orders,10,New customer,375.7,10,37.57,High spender,20.0,Regular customer,Female,Alabama,31,2/17/2019,3,married,40423,South,Regular-activity,Middle-aged,Low income,Middle-aged Low income,Middle-aged married,Non-alcohol consumer,Middle-aged Non-alcohol consumer
2,473747,1,3,3,12,21.0,196,1,1,Soda,77,7,9.0,Mid-range product,Regularly busy,Slowest day,Most orders,10,New customer,375.7,10,37.57,High spender,20.0,Regular customer,Female,Alabama,31,2/17/2019,3,married,40423,South,Regular-activity,Middle-aged,Low income,Middle-aged Low income,Middle-aged married,Non-alcohol consumer,Middle-aged Non-alcohol consumer
3,2254736,1,4,4,7,29.0,196,1,1,Soda,77,7,9.0,Mid-range product,Least busy,Slowest day,Average orders,10,New customer,375.7,10,37.57,High spender,20.0,Regular customer,Female,Alabama,31,2/17/2019,3,married,40423,South,Regular-activity,Middle-aged,Low income,Middle-aged Low income,Middle-aged married,Non-alcohol consumer,Middle-aged Non-alcohol consumer
4,431534,1,5,4,15,28.0,196,1,1,Soda,77,7,9.0,Mid-range product,Least busy,Slowest day,Most orders,10,New customer,375.7,10,37.57,High spender,20.0,Regular customer,Female,Alabama,31,2/17/2019,3,married,40423,South,Regular-activity,Middle-aged,Low income,Middle-aged Low income,Middle-aged married,Non-alcohol consumer,Middle-aged Non-alcohol consumer


In [119]:
# 'Alcohol_flag' itself will not be used as a standalone profiling variable
# Delete 'Alcohol_flag' from the master dataframe to save memory and space

del ords_prods_reg_custs_merge['Alcohol_flag']

In [120]:
# Check the shape after deletion

ords_prods_reg_custs_merge.shape

(30964518, 39)

In [121]:
# Check the head after deletion

ords_prods_reg_custs_merge.head()

,order_id,user_id,order_sequence_number,order_days_of_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,prices,price_range_loc,busiest_day,Busiest_Days,busiest_period_of_day,max_order,loyalty_flag,total_ord_spend,total_ord_count,avg_ord_spend,spending_flag,median_frequency,frequency_flag,Gender,STATE,Age,date_joined,n_dependants,fam_status,income,Region,customer_activity,age_bucket,income_bucket,Age_income_level,Age_fam_status,Age_alcohol_level
0,2539329,1,1,2,8,0.0,196,1,0,Soda,77,7,9.0,Mid-range product,Regularly busy,Normal day,Average orders,10,New customer,375.7,10,37.57,High spender,20.0,Regular customer,Female,Alabama,31,2/17/2019,3,married,40423,South,Regular-activity,Middle-aged,Low income,Middle-aged Low income,Middle-aged married,Middle-aged Non-alcohol consumer
1,2398795,1,2,3,7,15.0,196,1,1,Soda,77,7,9.0,Mid-range product,Regularly busy,Slowest day,Average orders,10,New customer,375.7,10,37.57,High spender,20.0,Regular customer,Female,Alabama,31,2/17/2019,3,married,40423,South,Regular-activity,Middle-aged,Low income,Middle-aged Low income,Middle-aged married,Middle-aged Non-alcohol consumer
2,473747,1,3,3,12,21.0,196,1,1,Soda,77,7,9.0,Mid-range product,Regularly busy,Slowest day,Most orders,10,New customer,375.7,10,37.57,High spender,20.0,Regular customer,Female,Alabama,31,2/17/2019,3,married,40423,South,Regular-activity,Middle-aged,Low income,Middle-aged Low income,Middle-aged married,Middle-aged Non-alcohol consumer
3,2254736,1,4,4,7,29.0,196,1,1,Soda,77,7,9.0,Mid-range product,Least busy,Slowest day,Average orders,10,New customer,375.7,10,37.57,High spender,20.0,Regular customer,Female,Alabama,31,2/17/2019,3,married,40423,South,Regular-activity,Middle-aged,Low income,Middle-aged Low income,Middle-aged married,Middle-aged Non-alcohol consumer
4,431534,1,5,4,15,28.0,196,1,1,Soda,77,7,9.0,Mid-range product,Least busy,Slowest day,Most orders,10,New customer,375.7,10,37.57,High spender,20.0,Regular customer,Female,Alabama,31,2/17/2019,3,married,40423,South,Regular-activity,Middle-aged,Low income,Middle-aged Low income,Middle-aged married,Middle-aged Non-alcohol consumer


## 04.04 Creating a profiling variable: Pet ownership by family status

CP: This is to create a profiling variable based on the pet ownership by the user's family status (e.g., Single pet owner, etc.)

In [122]:
# Identify the customers with pet related purchases vs not by creating a subset
# Create a subset to only include user id , order id, and department id
# Filter the subset by the rows containing only pet
# Remove the duplicated user ids from the filtered subset and assign value 'Pet owner' to a new column

user_pet = subset_dept_cat[subset_dept_cat['department_id'] == 8]
user_pet_no_dup = user_pet[['user_id']].drop_duplicates()
user_pet_no_dup['Pet_flag'] = 'Pet owner'

In [123]:
# Check the shape of the pet-related purchaser dataframe

user_pet_no_dup.shape

(13175, 2)

In [124]:
# Check the head of the pet-related purchase dataframe

user_pet_no_dup.head()

,user_id,Pet_flag
1495,109,Pet owner
2791,290,Pet owner
5196,709,Pet owner
6395,1137,Pet owner
6813,1306,Pet owner


In [125]:
# Check one sample user to see whether the person actually purchased pet-related items from the master dataframe

ords_prods_reg_custs_merge[(ords_prods_reg_custs_merge['user_id'] == 109) & (ords_prods_reg_custs_merge['department_id'] == 8)]

,order_id,user_id,order_sequence_number,order_days_of_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,prices,price_range_loc,busiest_day,Busiest_Days,busiest_period_of_day,max_order,loyalty_flag,total_ord_spend,total_ord_count,avg_ord_spend,spending_flag,median_frequency,frequency_flag,Gender,STATE,Age,date_joined,n_dependants,fam_status,income,Region,customer_activity,age_bucket,income_bucket,Age_income_level,Age_fam_status,Age_alcohol_level
1495,659764,109,4,2,5,20.0,36273,9,0,Scoopable Scented Clumping Cat Litter,41,8,10.1,Mid-range product,Regularly busy,Normal day,Fewest orders,6,New customer,403.7,6,67.283333,High spender,20.0,Regular customer,Female,Connecticut,67,7/29/2018,1,married,41805,Northeast,Regular-activity,Middle-aged,Low income,Middle-aged Low income,Middle-aged married,Middle-aged Alcohol consumer
1496,3116901,109,5,0,7,26.0,36273,3,1,Scoopable Scented Clumping Cat Litter,41,8,10.1,Mid-range product,Busiest day,Busiest day,Average orders,6,New customer,403.7,6,67.283333,High spender,20.0,Regular customer,Female,Connecticut,67,7/29/2018,1,married,41805,Northeast,Regular-activity,Middle-aged,Low income,Middle-aged Low income,Middle-aged married,Middle-aged Alcohol consumer


In [126]:
# Merge the filtered subset of pet supply consumers to the master dataframe using merge function

ords_prods_reg_custs_merge = pd.merge(ords_prods_reg_custs_merge, user_pet_no_dup, on = 'user_id', how = 'left', indicator = False)

In [127]:
# Check the value counts of the new column added to the master dataframe

ords_prods_reg_custs_merge['Pet_flag'].value_counts(dropna = False)

NaN          27513167
Pet owner     3451351
Name: Pet_flag, dtype: int64

In [128]:
# Also check whether the number of the unique pet owner equal to the one from the filtered subset

ords_prods_reg_custs_merge.groupby('Pet_flag')['user_id'].nunique()

Pet_flag
Pet owner    13175
Name: user_id, dtype: int64

In [129]:
# For the unmatched rows in the Pet_flag column, assign value of Non-alcohol consumer using loc function

ords_prods_reg_custs_merge.loc[ords_prods_reg_custs_merge['Pet_flag'].isnull(), 'Pet_flag'] = 'Non-pet owner'

In [130]:
# Check the value counts again

ords_prods_reg_custs_merge['Pet_flag'].value_counts(dropna = False)

Non-pet owner    27513167
Pet owner         3451351
Name: Pet_flag, dtype: int64

In [131]:
# Check the value categories and counts of the existing column to be combined in creation of a profiling variable

ords_prods_reg_custs_merge['fam_status'].value_counts(dropna = False)

married                             21743700
single                               5094375
divorced/widowed                     2645271
living with parents and siblings     1481172
Name: fam_status, dtype: int64

In [132]:
# Summarize the 'fam_status' into two main categories and create a new column 'household' using loc function

ords_prods_reg_custs_merge.loc[(ords_prods_reg_custs_merge['fam_status'] =='married') | (ords_prods_reg_custs_merge['fam_status'] =='living with parents and siblings'), 'household'] = 'Multi-member family'

In [133]:
# Summarize the 'fam_status' into two main categories and create a new column 'household' using loc function

ords_prods_reg_custs_merge.loc[(ords_prods_reg_custs_merge['fam_status'] =='single') | (ords_prods_reg_custs_merge['fam_status'] =='divorced/widowed'), 'household'] = 'Single-member family'

In [134]:
# Check the value counts of the 'household' column

ords_prods_reg_custs_merge['household'].value_counts(dropna = False)

Multi-member family     23224872
Single-member family     7739646
Name: household, dtype: int64

In [135]:
# Concatenate 'household' and 'Pet_flag' to create a new profile variable

ords_prods_reg_custs_merge['Pet_household'] = ords_prods_reg_custs_merge['household']+' '+ords_prods_reg_custs_merge['Pet_flag']

In [136]:
# Change the data type of the concatenated field into str

ords_prods_reg_custs_merge['Pet_household'] = ords_prods_reg_custs_merge['Pet_household'].astype('str')

In [137]:
# Check the shape of the appended dataframe

ords_prods_reg_custs_merge.shape

(30964518, 42)

In [138]:
# Check the head of the appended dataframe

ords_prods_reg_custs_merge.head()

,order_id,user_id,order_sequence_number,order_days_of_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,prices,price_range_loc,busiest_day,Busiest_Days,busiest_period_of_day,max_order,loyalty_flag,total_ord_spend,total_ord_count,avg_ord_spend,spending_flag,median_frequency,frequency_flag,Gender,STATE,Age,date_joined,n_dependants,fam_status,income,Region,customer_activity,age_bucket,income_bucket,Age_income_level,Age_fam_status,Age_alcohol_level,Pet_flag,household,Pet_household
0,2539329,1,1,2,8,0.0,196,1,0,Soda,77,7,9.0,Mid-range product,Regularly busy,Normal day,Average orders,10,New customer,375.7,10,37.57,High spender,20.0,Regular customer,Female,Alabama,31,2/17/2019,3,married,40423,South,Regular-activity,Middle-aged,Low income,Middle-aged Low income,Middle-aged married,Middle-aged Non-alcohol consumer,Non-pet owner,Multi-member family,Multi-member family Non-pet owner
1,2398795,1,2,3,7,15.0,196,1,1,Soda,77,7,9.0,Mid-range product,Regularly busy,Slowest day,Average orders,10,New customer,375.7,10,37.57,High spender,20.0,Regular customer,Female,Alabama,31,2/17/2019,3,married,40423,South,Regular-activity,Middle-aged,Low income,Middle-aged Low income,Middle-aged married,Middle-aged Non-alcohol consumer,Non-pet owner,Multi-member family,Multi-member family Non-pet owner
2,473747,1,3,3,12,21.0,196,1,1,Soda,77,7,9.0,Mid-range product,Regularly busy,Slowest day,Most orders,10,New customer,375.7,10,37.57,High spender,20.0,Regular customer,Female,Alabama,31,2/17/2019,3,married,40423,South,Regular-activity,Middle-aged,Low income,Middle-aged Low income,Middle-aged married,Middle-aged Non-alcohol consumer,Non-pet owner,Multi-member family,Multi-member family Non-pet owner
3,2254736,1,4,4,7,29.0,196,1,1,Soda,77,7,9.0,Mid-range product,Least busy,Slowest day,Average orders,10,New customer,375.7,10,37.57,High spender,20.0,Regular customer,Female,Alabama,31,2/17/2019,3,married,40423,South,Regular-activity,Middle-aged,Low income,Middle-aged Low income,Middle-aged married,Middle-aged Non-alcohol consumer,Non-pet owner,Multi-member family,Multi-member family Non-pet owner
4,431534,1,5,4,15,28.0,196,1,1,Soda,77,7,9.0,Mid-range product,Least busy,Slowest day,Most orders,10,New customer,375.7,10,37.57,High spender,20.0,Regular customer,Female,Alabama,31,2/17/2019,3,married,40423,South,Regular-activity,Middle-aged,Low income,Middle-aged Low income,Middle-aged married,Middle-aged Non-alcohol consumer,Non-pet owner,Multi-member family,Multi-member family Non-pet owner


In [139]:
# 'Pet_flag' and 'household' will not be used as standalone profiling variables
# Delete them from the master dataframe to save memory and space

ords_prods_reg_custs_merge['Pet_flag']
ords_prods_reg_custs_merge['household']

0            Multi-member family
1            Multi-member family
2            Multi-member family
3            Multi-member family
4            Multi-member family
                    ...         
30964513    Single-member family
30964514    Single-member family
30964515    Single-member family
30964516    Single-member family
30964517    Single-member family
Name: household, Length: 30964518, dtype: object

In [140]:
# Check the shape after deletion

ords_prods_reg_custs_merge.shape

(30964518, 42)

In [141]:
# Check the head after deletion

ords_prods_reg_custs_merge.head()

,order_id,user_id,order_sequence_number,order_days_of_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,prices,price_range_loc,busiest_day,Busiest_Days,busiest_period_of_day,max_order,loyalty_flag,total_ord_spend,total_ord_count,avg_ord_spend,spending_flag,median_frequency,frequency_flag,Gender,STATE,Age,date_joined,n_dependants,fam_status,income,Region,customer_activity,age_bucket,income_bucket,Age_income_level,Age_fam_status,Age_alcohol_level,Pet_flag,household,Pet_household
0,2539329,1,1,2,8,0.0,196,1,0,Soda,77,7,9.0,Mid-range product,Regularly busy,Normal day,Average orders,10,New customer,375.7,10,37.57,High spender,20.0,Regular customer,Female,Alabama,31,2/17/2019,3,married,40423,South,Regular-activity,Middle-aged,Low income,Middle-aged Low income,Middle-aged married,Middle-aged Non-alcohol consumer,Non-pet owner,Multi-member family,Multi-member family Non-pet owner
1,2398795,1,2,3,7,15.0,196,1,1,Soda,77,7,9.0,Mid-range product,Regularly busy,Slowest day,Average orders,10,New customer,375.7,10,37.57,High spender,20.0,Regular customer,Female,Alabama,31,2/17/2019,3,married,40423,South,Regular-activity,Middle-aged,Low income,Middle-aged Low income,Middle-aged married,Middle-aged Non-alcohol consumer,Non-pet owner,Multi-member family,Multi-member family Non-pet owner
2,473747,1,3,3,12,21.0,196,1,1,Soda,77,7,9.0,Mid-range product,Regularly busy,Slowest day,Most orders,10,New customer,375.7,10,37.57,High spender,20.0,Regular customer,Female,Alabama,31,2/17/2019,3,married,40423,South,Regular-activity,Middle-aged,Low income,Middle-aged Low income,Middle-aged married,Middle-aged Non-alcohol consumer,Non-pet owner,Multi-member family,Multi-member family Non-pet owner
3,2254736,1,4,4,7,29.0,196,1,1,Soda,77,7,9.0,Mid-range product,Least busy,Slowest day,Average orders,10,New customer,375.7,10,37.57,High spender,20.0,Regular customer,Female,Alabama,31,2/17/2019,3,married,40423,South,Regular-activity,Middle-aged,Low income,Middle-aged Low income,Middle-aged married,Middle-aged Non-alcohol consumer,Non-pet owner,Multi-member family,Multi-member family Non-pet owner
4,431534,1,5,4,15,28.0,196,1,1,Soda,77,7,9.0,Mid-range product,Least busy,Slowest day,Most orders,10,New customer,375.7,10,37.57,High spender,20.0,Regular customer,Female,Alabama,31,2/17/2019,3,married,40423,South,Regular-activity,Middle-aged,Low income,Middle-aged Low income,Middle-aged married,Middle-aged Non-alcohol consumer,Non-pet owner,Multi-member family,Multi-member family Non-pet owner


## 04.05 Creating a profiling variable: Family status and number of dependants

CP: This is to create a profiling variable to analyze the consumer buying power based on the family status and the number of dependants that users have (e.g., Single income with kids, etc.)

In [142]:
# Review the variable categories and counts in preparation for the profiling variable creation 

ords_prods_reg_custs_merge['fam_status'].value_counts(dropna = False)

married                             21743700
single                               5094375
divorced/widowed                     2645271
living with parents and siblings     1481172
Name: fam_status, dtype: int64

In [143]:
# Review the variable categories and counts in preparation for the profiling variable creation 

ords_prods_reg_custs_merge['n_dependants'].value_counts(dropna = False)

3    7772514
0    7739646
2    7733261
1    7719097
Name: n_dependants, dtype: int64

In [144]:
# Create a new column and assign variables based on the critera using loc function

ords_prods_reg_custs_merge.loc[((ords_prods_reg_custs_merge['fam_status'] == 'single') | (ords_prods_reg_custs_merge['fam_status'] == 'divorced/widowed')) & (ords_prods_reg_custs_merge['n_dependants'] == 0), 'buying_power'] = 'Single income with no kids'
ords_prods_reg_custs_merge.loc[((ords_prods_reg_custs_merge['fam_status'] == 'single') | (ords_prods_reg_custs_merge['fam_status'] == 'divorced/widowed')) & (ords_prods_reg_custs_merge['n_dependants'] == 1), 'buying_power'] = 'Single income with single kid'
ords_prods_reg_custs_merge.loc[((ords_prods_reg_custs_merge['fam_status'] == 'single') | (ords_prods_reg_custs_merge['fam_status'] == 'divorced/widowed')) & (ords_prods_reg_custs_merge['n_dependants'] > 1), 'buying_power'] = 'Single income with multiple kids'
ords_prods_reg_custs_merge.loc[(ords_prods_reg_custs_merge['fam_status'] == 'married') & (ords_prods_reg_custs_merge['n_dependants'] == 0), 'buying_power'] = 'Dual income with no kids'
ords_prods_reg_custs_merge.loc[(ords_prods_reg_custs_merge['fam_status'] == 'married') & (ords_prods_reg_custs_merge['n_dependants'] == 1), 'buying_power'] = 'Dual income with single kid'
ords_prods_reg_custs_merge.loc[(ords_prods_reg_custs_merge['fam_status'] == 'married') & (ords_prods_reg_custs_merge['n_dependants'] > 1), 'buying_power'] = 'Dual income with multiple kids'
ords_prods_reg_custs_merge.loc[ords_prods_reg_custs_merge['fam_status'] == 'living with parents and siblings', 'buying_power'] = 'Dependant of someone'


In [145]:
# Check the value counts of the appended column

ords_prods_reg_custs_merge['buying_power'].value_counts()

Dual income with multiple kids    14532648
Single income with no kids         7739646
Dual income with single kid        7211052
Dependant of someone               1481172
Name: buying_power, dtype: int64

In [146]:
# Check the head of the revised master dataframe

ords_prods_reg_custs_merge.head()

,order_id,user_id,order_sequence_number,order_days_of_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,prices,price_range_loc,busiest_day,Busiest_Days,busiest_period_of_day,max_order,loyalty_flag,total_ord_spend,total_ord_count,avg_ord_spend,spending_flag,median_frequency,frequency_flag,Gender,STATE,Age,date_joined,n_dependants,fam_status,income,Region,customer_activity,age_bucket,income_bucket,Age_income_level,Age_fam_status,Age_alcohol_level,Pet_flag,household,Pet_household,buying_power
0,2539329,1,1,2,8,0.0,196,1,0,Soda,77,7,9.0,Mid-range product,Regularly busy,Normal day,Average orders,10,New customer,375.7,10,37.57,High spender,20.0,Regular customer,Female,Alabama,31,2/17/2019,3,married,40423,South,Regular-activity,Middle-aged,Low income,Middle-aged Low income,Middle-aged married,Middle-aged Non-alcohol consumer,Non-pet owner,Multi-member family,Multi-member family Non-pet owner,Dual income with multiple kids
1,2398795,1,2,3,7,15.0,196,1,1,Soda,77,7,9.0,Mid-range product,Regularly busy,Slowest day,Average orders,10,New customer,375.7,10,37.57,High spender,20.0,Regular customer,Female,Alabama,31,2/17/2019,3,married,40423,South,Regular-activity,Middle-aged,Low income,Middle-aged Low income,Middle-aged married,Middle-aged Non-alcohol consumer,Non-pet owner,Multi-member family,Multi-member family Non-pet owner,Dual income with multiple kids
2,473747,1,3,3,12,21.0,196,1,1,Soda,77,7,9.0,Mid-range product,Regularly busy,Slowest day,Most orders,10,New customer,375.7,10,37.57,High spender,20.0,Regular customer,Female,Alabama,31,2/17/2019,3,married,40423,South,Regular-activity,Middle-aged,Low income,Middle-aged Low income,Middle-aged married,Middle-aged Non-alcohol consumer,Non-pet owner,Multi-member family,Multi-member family Non-pet owner,Dual income with multiple kids
3,2254736,1,4,4,7,29.0,196,1,1,Soda,77,7,9.0,Mid-range product,Least busy,Slowest day,Average orders,10,New customer,375.7,10,37.57,High spender,20.0,Regular customer,Female,Alabama,31,2/17/2019,3,married,40423,South,Regular-activity,Middle-aged,Low income,Middle-aged Low income,Middle-aged married,Middle-aged Non-alcohol consumer,Non-pet owner,Multi-member family,Multi-member family Non-pet owner,Dual income with multiple kids
4,431534,1,5,4,15,28.0,196,1,1,Soda,77,7,9.0,Mid-range product,Least busy,Slowest day,Most orders,10,New customer,375.7,10,37.57,High spender,20.0,Regular customer,Female,Alabama,31,2/17/2019,3,married,40423,South,Regular-activity,Middle-aged,Low income,Middle-aged Low income,Middle-aged married,Middle-aged Non-alcohol consumer,Non-pet owner,Multi-member family,Multi-member family Non-pet owner,Dual income with multiple kids


## 04.06 Creating a profiling variable: Shopping day time

CP: This is to create a profiling variable by combining their shopping day/hour to categorize customers.  

In [147]:
# Bucketing shopping hours by following:
# 20-23 & 0-3: Night
# 4-10: Morning
# 11-15: Afternoon
# 16-19: Evening
# 
# Bucketing shopping days by following:
# 0 & 1 (Saturday & Sunday): Weekend
# 2-6: Weekday
#
# Create 'order_time' column to indicate the day and hour the order was place (e.g., Weekday Morning) using loc

ords_prods_reg_custs_merge.loc[((ords_prods_reg_custs_merge['order_days_of_week'] == 0) | (ords_prods_reg_custs_merge['order_days_of_week'] == 1)) & ((ords_prods_reg_custs_merge['order_hour_of_day'] >= 20) | (ords_prods_reg_custs_merge['order_hour_of_day'] <= 3)), 'order_time'] = 'Weekend night'
ords_prods_reg_custs_merge.loc[((ords_prods_reg_custs_merge['order_days_of_week'] == 0) | (ords_prods_reg_custs_merge['order_days_of_week'] == 1)) & ((ords_prods_reg_custs_merge['order_hour_of_day'] >= 4) & (ords_prods_reg_custs_merge['order_hour_of_day'] <= 10)), 'order_time'] = 'Weekend morning'
ords_prods_reg_custs_merge.loc[((ords_prods_reg_custs_merge['order_days_of_week'] == 0) | (ords_prods_reg_custs_merge['order_days_of_week'] == 1)) & ((ords_prods_reg_custs_merge['order_hour_of_day'] >= 11) & (ords_prods_reg_custs_merge['order_hour_of_day'] <= 15)), 'order_time'] = 'Weekend afternoon'
ords_prods_reg_custs_merge.loc[((ords_prods_reg_custs_merge['order_days_of_week'] == 0) | (ords_prods_reg_custs_merge['order_days_of_week'] == 1)) & ((ords_prods_reg_custs_merge['order_hour_of_day'] >= 16) & (ords_prods_reg_custs_merge['order_hour_of_day'] <= 19)), 'order_time'] = 'Weekend evening'
ords_prods_reg_custs_merge.loc[((ords_prods_reg_custs_merge['order_days_of_week'] >= 2) & (ords_prods_reg_custs_merge['order_days_of_week'] <= 6)) & ((ords_prods_reg_custs_merge['order_hour_of_day'] >= 20) | (ords_prods_reg_custs_merge['order_hour_of_day'] <= 3)), 'order_time'] = 'Weekday night'
ords_prods_reg_custs_merge.loc[((ords_prods_reg_custs_merge['order_days_of_week'] >= 2) & (ords_prods_reg_custs_merge['order_days_of_week'] <= 6)) & ((ords_prods_reg_custs_merge['order_hour_of_day'] >= 4) & (ords_prods_reg_custs_merge['order_hour_of_day'] <= 10)), 'order_time'] = 'Weekday morning'
ords_prods_reg_custs_merge.loc[((ords_prods_reg_custs_merge['order_days_of_week'] >= 2) & (ords_prods_reg_custs_merge['order_days_of_week'] <= 6)) & ((ords_prods_reg_custs_merge['order_hour_of_day'] >= 11) & (ords_prods_reg_custs_merge['order_hour_of_day'] <= 15)), 'order_time'] = 'Weekday afternoon'
ords_prods_reg_custs_merge.loc[((ords_prods_reg_custs_merge['order_days_of_week'] >= 2) & (ords_prods_reg_custs_merge['order_days_of_week'] <= 6)) & ((ords_prods_reg_custs_merge['order_hour_of_day'] >= 16) & (ords_prods_reg_custs_merge['order_hour_of_day'] <= 19)), 'order_time'] = 'Weekday evening'

In [148]:
# Check the value counts of the column 'order_time' column created

ords_prods_reg_custs_merge['order_time'].value_counts()

Weekday afternoon    7927913
Weekday morning      4986073
Weekend afternoon    4825894
Weekday evening      4667786
Weekend morning      2965795
Weekend evening      2473273
Weekday night        2062457
Weekend night        1055327
Name: order_time, dtype: int64

In [149]:
# Check the head of the appended master dataframe for the accuracy verification

ords_prods_reg_custs_merge.head()

,order_id,user_id,order_sequence_number,order_days_of_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,prices,price_range_loc,busiest_day,Busiest_Days,busiest_period_of_day,max_order,loyalty_flag,total_ord_spend,total_ord_count,avg_ord_spend,spending_flag,median_frequency,frequency_flag,Gender,STATE,Age,date_joined,n_dependants,fam_status,income,Region,customer_activity,age_bucket,income_bucket,Age_income_level,Age_fam_status,Age_alcohol_level,Pet_flag,household,Pet_household,buying_power,order_time
0,2539329,1,1,2,8,0.0,196,1,0,Soda,77,7,9.0,Mid-range product,Regularly busy,Normal day,Average orders,10,New customer,375.7,10,37.57,High spender,20.0,Regular customer,Female,Alabama,31,2/17/2019,3,married,40423,South,Regular-activity,Middle-aged,Low income,Middle-aged Low income,Middle-aged married,Middle-aged Non-alcohol consumer,Non-pet owner,Multi-member family,Multi-member family Non-pet owner,Dual income with multiple kids,Weekday morning
1,2398795,1,2,3,7,15.0,196,1,1,Soda,77,7,9.0,Mid-range product,Regularly busy,Slowest day,Average orders,10,New customer,375.7,10,37.57,High spender,20.0,Regular customer,Female,Alabama,31,2/17/2019,3,married,40423,South,Regular-activity,Middle-aged,Low income,Middle-aged Low income,Middle-aged married,Middle-aged Non-alcohol consumer,Non-pet owner,Multi-member family,Multi-member family Non-pet owner,Dual income with multiple kids,Weekday morning
2,473747,1,3,3,12,21.0,196,1,1,Soda,77,7,9.0,Mid-range product,Regularly busy,Slowest day,Most orders,10,New customer,375.7,10,37.57,High spender,20.0,Regular customer,Female,Alabama,31,2/17/2019,3,married,40423,South,Regular-activity,Middle-aged,Low income,Middle-aged Low income,Middle-aged married,Middle-aged Non-alcohol consumer,Non-pet owner,Multi-member family,Multi-member family Non-pet owner,Dual income with multiple kids,Weekday afternoon
3,2254736,1,4,4,7,29.0,196,1,1,Soda,77,7,9.0,Mid-range product,Least busy,Slowest day,Average orders,10,New customer,375.7,10,37.57,High spender,20.0,Regular customer,Female,Alabama,31,2/17/2019,3,married,40423,South,Regular-activity,Middle-aged,Low income,Middle-aged Low income,Middle-aged married,Middle-aged Non-alcohol consumer,Non-pet owner,Multi-member family,Multi-member family Non-pet owner,Dual income with multiple kids,Weekday morning
4,431534,1,5,4,15,28.0,196,1,1,Soda,77,7,9.0,Mid-range product,Least busy,Slowest day,Most orders,10,New customer,375.7,10,37.57,High spender,20.0,Regular customer,Female,Alabama,31,2/17/2019,3,married,40423,South,Regular-activity,Middle-aged,Low income,Middle-aged Low income,Middle-aged married,Middle-aged Non-alcohol consumer,Non-pet owner,Multi-member family,Multi-member family Non-pet owner,Dual income with multiple kids,Weekday afternoon


In [150]:
# A customer may place orders during the different days/times
# In order to review what each customer's preferred order placing day/time
# The mode of 'order_time' for each customer
# Check the mode of 'order_time' by each customer using agg and pd.Series.mode function

ords_prods_reg_custs_merge.groupby(['user_id'])['order_time'].apply(lambda x: x.mode().iloc[0]).to_frame()

,order_time
user_id,
1,Weekday morning
2,Weekday morning
3,Weekend evening
4,Weekday afternoon
7,Weekday morning
...,...
206203,Weekend evening
206206,Weekday evening
206207,Weekday afternoon


In [151]:
# Check to see whether this calculation is correct by looking into one sample

ords_prods_reg_custs_merge[ords_prods_reg_custs_merge['user_id'] == 1]['order_time'].value_counts(dropna = False)

Weekday morning      29
Weekday afternoon    13
Weekend afternoon     6
Weekend evening       6
Weekend morning       5
Name: order_time, dtype: int64

In [152]:
# Create a dataframe of the modes of 'order_time' by 'user_id'

ord_time_pref = ords_prods_reg_custs_merge.groupby(['user_id'])['order_time'].apply(lambda x: x.mode().iloc[0])

In [153]:
# Reset the index after using groupby

ord_time_pref = ord_time_pref.reset_index()

In [154]:
# Verify the dataframe information

ord_time_pref.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 162627 entries, 0 to 162626
Data columns (total 2 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   user_id     162627 non-null  int64 
 1   order_time  162627 non-null  object
dtypes: int64(1), object(1)
memory usage: 2.5+ MB


In [155]:
# Check the value counts of 'order_time'

ord_time_pref['order_time'].value_counts()

Weekday afternoon    60112
Weekday evening      27005
Weekday morning      25697
Weekend afternoon    22094
Weekend morning       9025
Weekday night         8290
Weekend evening       7995
Weekend night         2409
Name: order_time, dtype: int64

In [156]:
# Reset the data type of 'order_time' to str

ord_time_pref['order_time'] = ord_time_pref['order_time'].astype('str')

In [157]:
# For the merging purpose, create another column called 'user_preferred_shop_time' in this new dataframe

ord_time_pref['user_preferred_shop_time'] = (ord_time_pref['order_time']+ ' shopper').astype('str')

In [158]:
# Display the modified dataframe

ord_time_pref

,user_id,order_time,user_preferred_shop_time
0,1,Weekday morning,Weekday morning shopper
1,2,Weekday morning,Weekday morning shopper
2,3,Weekend evening,Weekend evening shopper
3,4,Weekday afternoon,Weekday afternoon shopper
4,7,Weekday morning,Weekday morning shopper
...,...,...,...
162622,206203,Weekend evening,Weekend evening shopper
162623,206206,Weekday evening,Weekday evening shopper
162624,206207,Weekday afternoon,Weekday afternoon shopper
162625,206208,Weekday afternoon,Weekday afternoon shopper


In [159]:
# Check the value counts of the new column to see whether they can be grouped correctly

ord_time_pref['user_preferred_shop_time'].value_counts(dropna = False)

Weekday afternoon shopper    60112
Weekday evening shopper      27005
Weekday morning shopper      25697
Weekend afternoon shopper    22094
Weekend morning shopper       9025
Weekday night shopper         8290
Weekend evening shopper       7995
Weekend night shopper         2409
Name: user_preferred_shop_time, dtype: int64

In [160]:
# Delete 'order_time' column prior to merging to the master dataframe

del ord_time_pref['order_time']

In [161]:
# Merge 'ord_time_pref' into the master dataframe

ords_prods_reg_custs_merge = pd.merge(ords_prods_reg_custs_merge, ord_time_pref, on = 'user_id', how = 'left', indicator = False)

In [162]:
# Check the shape of the appended master dataframe

ords_prods_reg_custs_merge.shape

(30964518, 45)

In [163]:
# Check the head of the appended master dataframe

ords_prods_reg_custs_merge.head()

,order_id,user_id,order_sequence_number,order_days_of_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,prices,price_range_loc,busiest_day,Busiest_Days,busiest_period_of_day,max_order,loyalty_flag,total_ord_spend,total_ord_count,avg_ord_spend,spending_flag,median_frequency,frequency_flag,Gender,STATE,Age,date_joined,n_dependants,fam_status,income,Region,customer_activity,age_bucket,income_bucket,Age_income_level,Age_fam_status,Age_alcohol_level,Pet_flag,household,Pet_household,buying_power,order_time,user_preferred_shop_time
0,2539329,1,1,2,8,0.0,196,1,0,Soda,77,7,9.0,Mid-range product,Regularly busy,Normal day,Average orders,10,New customer,375.7,10,37.57,High spender,20.0,Regular customer,Female,Alabama,31,2/17/2019,3,married,40423,South,Regular-activity,Middle-aged,Low income,Middle-aged Low income,Middle-aged married,Middle-aged Non-alcohol consumer,Non-pet owner,Multi-member family,Multi-member family Non-pet owner,Dual income with multiple kids,Weekday morning,Weekday morning shopper
1,2398795,1,2,3,7,15.0,196,1,1,Soda,77,7,9.0,Mid-range product,Regularly busy,Slowest day,Average orders,10,New customer,375.7,10,37.57,High spender,20.0,Regular customer,Female,Alabama,31,2/17/2019,3,married,40423,South,Regular-activity,Middle-aged,Low income,Middle-aged Low income,Middle-aged married,Middle-aged Non-alcohol consumer,Non-pet owner,Multi-member family,Multi-member family Non-pet owner,Dual income with multiple kids,Weekday morning,Weekday morning shopper
2,473747,1,3,3,12,21.0,196,1,1,Soda,77,7,9.0,Mid-range product,Regularly busy,Slowest day,Most orders,10,New customer,375.7,10,37.57,High spender,20.0,Regular customer,Female,Alabama,31,2/17/2019,3,married,40423,South,Regular-activity,Middle-aged,Low income,Middle-aged Low income,Middle-aged married,Middle-aged Non-alcohol consumer,Non-pet owner,Multi-member family,Multi-member family Non-pet owner,Dual income with multiple kids,Weekday afternoon,Weekday morning shopper
3,2254736,1,4,4,7,29.0,196,1,1,Soda,77,7,9.0,Mid-range product,Least busy,Slowest day,Average orders,10,New customer,375.7,10,37.57,High spender,20.0,Regular customer,Female,Alabama,31,2/17/2019,3,married,40423,South,Regular-activity,Middle-aged,Low income,Middle-aged Low income,Middle-aged married,Middle-aged Non-alcohol consumer,Non-pet owner,Multi-member family,Multi-member family Non-pet owner,Dual income with multiple kids,Weekday morning,Weekday morning shopper
4,431534,1,5,4,15,28.0,196,1,1,Soda,77,7,9.0,Mid-range product,Least busy,Slowest day,Most orders,10,New customer,375.7,10,37.57,High spender,20.0,Regular customer,Female,Alabama,31,2/17/2019,3,married,40423,South,Regular-activity,Middle-aged,Low income,Middle-aged Low income,Middle-aged married,Middle-aged Non-alcohol consumer,Non-pet owner,Multi-member family,Multi-member family Non-pet owner,Dual income with multiple kids,Weekday afternoon,Weekday morning shopper


In [164]:
# Delete 'ord_time_pref' dataframe upon verifying its merge to save memory

del ord_time_pref

# 05. Exporting data

In [165]:
# Check the shape of the manipulated data so far

ords_prods_reg_custs_merge.shape

(30964518, 45)

In [166]:
# Export the manipulated data
# This notebook will be closed and the new notebook will be created to continue on the analysis

ords_prods_reg_custs_merge.to_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'orders_products_all.pkl'))